#### New to Plotly?
Plotly's Python library is free and open source! [Get started](https://plot.ly/python/getting-started/) by dowloading the client and [reading the primer](https://plot.ly/python/getting-started/).
<br>You can set up Plotly to work in [online](https://plot.ly/python/getting-started/#initialization-for-online-plotting) or [offline](https://plot.ly/python/getting-started/#initialization-for-offline-plotting) mode, or in [jupyter notebooks](https://plot.ly/python/getting-started/#start-plotting-online).
<br>We also have a quick-reference [cheatsheet](https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf) (new!) to help you get started!


## Heatmaps of projections of a 3d surface onto planes perpendicular to the z, x, respectively y-direction ##

Usually surfaces in the 3d space are colored with  a colormap associated to the normalized range of the z coordinates of points on that surface.
Recently, Plotly devised a method to color a surface according to a custom color function.

Namely, if $x, y, z$ are numpy arrays of shape (m, n), defined by a discretization (via a meshgrid) of a surface z=f(x,y) or in parametric form, $x=x(u,v), y=y(u,v), z=z(u,v)$, then a custom function, `Color(x,y,z)`, returns a numpy array `C`, of the same shape as z, and the surface is colored by a colormap, according to the values in `C`.

This method allows to project a surface onto planes perpendicular to the z, x or y-direction in the 3d space
and interpret the projection as a planar surface colored according to the z, x or y value at each point
of the coresponding plane.

In [1]:
import numpy as np

Define the surface and its discretization:

In [2]:
xx=np.linspace(-3.5, 3.5, 100)
yy=np.linspace(-3.5, 3.5, 100)
x,y=np.meshgrid(xx, yy)
z=np.exp(-(x-1)**2-y**2)-10*(x**3+y**4-x/5)*np.exp(-(x**2+y**2))

The surface will be colored according to its normalized z-values, using the following colorscale:

In [3]:
colorscale=[[0.0, 'rgb(20,29,67)'],
           [0.1, 'rgb(28,76,96)'],
           [0.2, 'rgb(16,125,121)'],
           [0.3, 'rgb(92,166,133)'],
           [0.4, 'rgb(182,202,175)'],
           [0.5, 'rgb(253,245,243)'],
           [0.6, 'rgb(230,183,162)'],
           [0.7, 'rgb(211,118,105)'],
           [0.8, 'rgb(174,63,95)'],
           [0.9, 'rgb(116,25,93)'],
           [1.0, 'rgb(51,13,53)']]

In [4]:
import plotly.plotly as py
from plotly.graph_objs import *

Define the hover text for the surface:

In [5]:
textz=[['x: '+'{:0.5f}'.format(x[i][j])+'<br>y: '+'{:0.5f}'.format(y[i][j])+
        '<br>z: '+'{:0.5f}'.format(z[i][j]) for j in range(z.shape[1])] for i in range(z.shape[0])]

In [6]:
trace1= Surface(z=z,
                x=x,
                y=y, 
                colorscale=colorscale,
                text=textz,
                hoverinfo='text',
                )

Set the plot layout:

In [7]:
axis = dict(
showbackground=True, 
backgroundcolor="rgb(230, 230,230)", 
showgrid=False,    
zeroline=False,  
showline=False   
    )

ztickvals=range(-6,4)
layout = Layout(title="Projections of a surface onto coordinate planes" , 
                autosize=False,
                width=700,
                height=600,
                scene=Scene(xaxis=XAxis(axis, range=[-3.5, 3.5]),
                            yaxis=YAxis(axis, range=[-3.5, 3.5]),
                            zaxis=ZAxis(axis , tickvals=ztickvals),
                            aspectratio=dict(x=1,
                                             y=1,
                                             z=0.95
                                            )
                           )
                )

The surface projections will be plotted in the planes of equations
`Z=np.min(z)-2`, `X=np.min(xx)`, respectively `Y=np.min(yy)`.

We define a discretization of each such a plane, associated to the initial meshgrids for the complementary variables:

In [8]:
z_offset=(np.min(z)-2)*np.ones(z.shape)#
x_offset=np.min(xx)*np.ones(z.shape)
y_offset=np.min(yy)*np.ones(z.shape)

Define the color functions and the color numpy arrays, `C_z`, `C_x`, `C_y`, corresponding to  each plane:

In [9]:
color_z=lambda x, y, z: z#color function for the projection in the z-direction
C_z=color_z(x,y,z)
color_x=lambda x, y, z: x
C_x=color_x(x,y,z)
color_y=lambda x, y, z: y
C_y=color_y(x,y,z)

Define the 3-tuples of coordinates to be displayed at hovering the mouse over the projections. 
The first two coordinates give the position in the projection plane, whereas the third one  is used 
for assigning the color, just in the same way the coordinate z is used for the z-direction projection.

In [10]:
textx=[['y: '+'{:0.5f}'.format(y[i][j])+'<br>z: '+'{:0.5f}'.format(z[i][j])+
        '<br>x: '+'{:0.5f}'.format(x[i][j]) for j in range(z.shape[1])]  for i in range(z.shape[0])]
texty=[['x: '+'{:0.5f}'.format(x[i][j])+'<br>z: '+'{:0.5f}'.format(z[i][j]) +
        '<br>y: '+'{:0.5f}'.format(y[i][j]) for j in range(z.shape[1])] for i in range(z.shape[0])]  


In [11]:
tracex =Surface(z=z,
                x=x_offset,
                y=y,
                colorscale=colorscale,
                showlegend=False,
                showscale=False,
                surfacecolor=C_x,
                text=textx,
                hoverinfo='text'
               )
tracey =Surface(z=z,
                x=x,
                y=y_offset,
                colorscale=colorscale,
                showlegend=False,
                showscale=False,
                surfacecolor=C_y,
                text=texty,
                hoverinfo='text'
               )
tracez =Surface(z=z_offset,
                x=x,
                y=y,
                colorscale=colorscale,
                showlegend=False,
                showscale=False,
                surfacecolor=C_z,
                text=textz,
                hoverinfo='text'
               )

In [12]:
data=Data([trace1, tracex, tracey, tracez])

In [13]:
fig = Figure(data=data, layout=layout)

In [14]:
py.iplot(fig)

In [2]:
from IPython.display import display, HTML

display(HTML('<link href="//fonts.googleapis.com/css?family=Open+Sans:600,400,300,200|Inconsolata|Ubuntu+Mono:400,700rel="stylesheet" type="text/css" />'))
display(HTML('<link rel="stylesheet" type="text/csshref="http://help.plot.ly/documentation/all_static/css/ipython-notebook-custom.css">'))

! pip install git+https://github.com/plotly/publisher.git --upgrade

import publisher
publisher.publish(
    'Plotly-project-3d-onto-a-plane.ipynb', 'python/2d-projection-of-3d-surface/', 'Projection of 3D Surface | plotly',
    'How to project 3D Surface plots in 2D with Plotly.',
    title = '2D Projection of 3D surface | plotly',
    name = 'Projection of 3D surface',
    has_thumbnail='true', thumbnail='thumbnail/projection-3d.jpg', 
    language='python', page_type='example_index',
    display_as='3d_charts', order=18)  


Downloading/unpacking git+https://github.com/plotly/publisher.git
  Cloning https://github.com/plotly/publisher.git to /tmp/pip-U1KQcu-build
  Running setup.py (path:/tmp/pip-U1KQcu-build/setup.py) egg_info for package from git+https://github.com/plotly/publisher.git
    
    no previously-included directories found matching 'dist'
  Running setup.py install for publisher
    
    no previously-included directories found matching 'dist'
Cleaning up...
